# 라이브러리 import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import rc # 폰트
from datetime import datetime
import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
# datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

pd.set_option ('display.max_rows', 10)



# path = 'C:\\Users\\wldbs\\Desktop\\kart-game-project\\[data]_crawling\\'

# 크롤링 데이터 전처리

In [5]:
crawl = pd.read_csv(path + 'scraping-rawdata.csv')

In [6]:
# merge를 위해 변수명 변경

crawl.columns = ['map_name', '태그', '난이도', '랩', '트랙 길이', '진행 방향', '트랙 분류', 'AI 주행', '첫 등장일','리그 트랙', '라이센스', '약칭']

In [7]:
# 차이나 북경 경기장의 경우 난이도, 랩 잘못 입력되어있음. 아예 삭제하는 게 낫겠다는 판단
drop_data = crawl[crawl['난이도'].isnull()].index

crawl.drop(drop_data, inplace = True)

In [8]:
# 결측치 대치 (missing 이라는 문자값으로 입력되게 함)

crawl = crawl.fillna('missing')

In [9]:
# 필요 없는 컬럼 제거

crawl = crawl.drop(['태그', '트랙 분류', '리그 트랙', '라이센스', '약칭', 'AI 주행'], axis = 1)

In [10]:
# 난이도 문자열 - 숫자열로 변경

crawl['난이도'] = crawl.난이도.str.split('[').str[0]
crawl['난이도'] = crawl.난이도.str.split(' ').str[0]

crawl.난이도.unique()

crawl['난이도'].replace( {'●○○○○' : 1,
                          '●●○○○' : 2,
                          '●●●○○' : 3,
                          '●●●●○' : 4,
                          '●●●●●' : 5,
                          '●●○○○○' : 2,
                          '●●●●●○' : 4,
                          '●●●○○○' : 3,
                          '●●●●●●' : 5
                          }, inplace = True)

In [11]:
# 랩 문자열 - 숫자열로 변경

crawl['랩'] = crawl.랩.str.split('바').str[0]
crawl['랩']= crawl.랩.astype('int')

In [12]:
# 트랙 길이 문자열 - 숫자열로 변경

crawl['트랙 길이'].unique()

crawl['트랙 길이'] = crawl['트랙 길이'].str.split('k').str[0]
crawl['트랙 길이'] = crawl['트랙 길이'].str.split(' ').str[0]
crawl['트랙 길이'].replace( {'???' : 0 }, inplace = True)

crawl['트랙 길이'] = crawl['트랙 길이'].astype('float')

In [13]:
# 범주형 변수 따로 변환하지 않기로 함
crawl['진행 방향'].unique()

array(['반시계방향', '시계방향', '이중 8자형', '단선형', '8자형', '왕복형', '워프형', '스피드전',
       '삼중 8자형', '이중 시계방향', '이중 반시계방향', '복합 8자형'], dtype=object)

In [14]:
# # 날짜형 변환

crawl['년'] = crawl['첫 등장일'].str.split(' ').str[0]
crawl['월'] = crawl['첫 등장일'].str.split(' ').str[1]
crawl['일'] = crawl['첫 등장일'].str.split(' ').str[2]

crawl['년'] = crawl['년'].str.strip('년').str[0:4]
crawl['월'] = crawl['월'].str.split('월').str[0]
crawl['일'] = crawl['일'].str.split('일').str[0]

crawl['월'] = crawl['월'].str.rjust(width=2,fillchar='0')        # rlpad
crawl['일'] = crawl['일'].str.rjust(width=2,fillchar='0')        # rlpad

In [15]:
crawl['년'].unique()

array(['2004', '2006', '2007', '2009', '2016', '2005', '2019', '2010',
       '2015', '2020', '2008', 'miss', '2017', '2012', '2011', '2021',
       '2013', '2014', '200-', '2022', '-', '2018'], dtype=object)

In [16]:
# 년 변환
crawl['년'].replace( {'200-' : np.NaN,
                      'miss' : np.NaN,
                      '-'    : np.NaN}, inplace = True)

# 월 변환
crawl['월'].replace( {'0-' : np.NaN,
                      '0?' : np.NaN
                      }, inplace = True)

# 일 변환
crawl['일'].replace( {'0-' : np.NaN,
                      '0?' : np.NaN
                      }, inplace = True)

In [17]:
# 출시일 컬럼 생성

crawl['출시일'] = crawl['년'] + crawl['월'] + crawl['일']

In [21]:
# crawl['출시일'].unique()

In [19]:
# 날짜형으로 변경

crawl['출시일'] = pd.to_datetime(crawl['출시일'])

In [20]:
# 출시일의 nan 값 0으로 변경

crawl = crawl.fillna('0')

In [22]:
# 중복 데이터 있는지 확인
# 30개 행의 map_name이 중복되고 있음

crawl.map_name.nunique()

304

In [23]:
# 중복 내용이 모든 컬럼에 대해 중복인지 확인
# 모든 컬럼에 대한 중복이 맞음
crawl.loc[crawl.map_name.duplicated()==True].sort_values(by = 'map_name', ascending = False)

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일
58,빌리지 파괴된 시계탑,2,3,6.3,8자형,2010년 12월 30일,2010,12,30,2010-12-30 00:00:00
74,빌리지 파괴된 손가락,3,3,6.0,시계방향,2011년 8월 25일,2011,08,25,2011-08-25 00:00:00
65,빌리지 크레이지 운하,2,3,4.8,시계방향,2015년 12월 23일,2015,12,23,2015-12-23 00:00:00
75,빌리지 지우펀,4,1,5.5,단선형,2012년 9월 6일,2012,09,06,2012-09-06 00:00:00
71,빌리지 익스트림 경기장,3,2,7.4,시계방향,2008년 5월 22일,2008,05,22,2008-05-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...
63,[R] 빌리지 수로,2,3,6.0,8자형,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00
79,[R] 빌리지 손가락,3,3,6.0,반시계방향,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00
78,[R] 빌리지 붐힐터널,4,1,5.3,반시계방향,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00
61,[R] 빌리지 두개의 관문,1,3,6.9,반시계방향,2005년 4월 14일,2005,04,14,2005-04-14 00:00:00


In [24]:
# 중복 제거
crawl = crawl.drop_duplicates()

# json 기반 trackId와 theme table merge하기

In [27]:
compare_table = pd.read_csv(path + "00-json-metadata-trackId-with-map-name.csv", encoding = 'cp949')


# 스피드전에 대한 테마만 존재
theme_table = pd.read_csv(path + '00-track-theme.csv')

In [2]:
# 카멜롯 외곽 순찰로 이름으로 trackId가 두 개 있음 
# 확인해보니, 3일치 데이터에서는 4cdbb5 ~ trackId만 존재함
# 9f583 ~ ID는 과거의 아이디였을 것으로 추정됨
# 삭제할지, (구) 를 붙여 남겨둘지는 고민필요

compare_table['duplicated'] = compare_table['map_name'].duplicated()
compare_table.loc[compare_table['duplicated'] == True]

compare_table['trackId'].loc[compare_table.map_name == '카멜롯 외곽 순찰로'].to_list()

In [1]:
# 카멜롯 외곽 순찰로의 두 trackId 중 9f~ id는 과거 id로 판단하고, 삭제하기로 결정
# map_name 으로 join하기 때문에 (구) 를 붙여 이름을 바꾸더라도, 교집합으로 처리되지 않아 삭제될 것으로 예상되었기 때문

drop_data = compare_table[compare_table['trackId'] == '9f5830793101bb00f5b1889745bef379020e668e8b969b53906113ec67a25a86'].index

compare_table.drop(drop_data, inplace = True)

# 필요 없는 'duplicated' 컬럼 제거
compare_table = compare_table.drop(['duplicated'], axis = 1)

In [ ]:
# trackId_name 테이블과 theme_table merge
# meta data 기반 테이블을 기준으로 merge 하였음
# 여기서 theme 나오지 않는 map_name의 경우 나중에 수기로 붙이기로 결정

new_meta_table = pd.merge(theme_table, compare_table, left_on = '트랙이름', right_on = 'map_name', how = 'right')

new_meta_table.loc[new_meta_table['트랙이름'] == new_meta_table['map_name'], 'result'] = 'equal'

In [ ]:
# 중복, 필요 없는 컬럼 제거

new_meta_table = new_meta_table.drop(['트랙이름', 'result'], axis = 1)

In [ ]:
# 저장

new_meta_table.to_csv(path + 'scraping.csv', index=False, encoding = 'utf-8')